# Random Forest Modelling
- Data
    - Domain 1
    - Domain 2
    - Domain 1 & 2

- Data Processing
    - Sequence
    - Bag of Words
        - with n-gram
    - TFIDF
        - with n-gram

- Parameters
    - Tree Depth

- Validation
    - 10-fold cross validation
    
- Visualisation
    - Accuracy Graph
    - Confusion Matrix


In [56]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import numpy as np
import json

## 1. Load Data

In [57]:
def load_data(index):
    text = []
    label = []

    if index == 1 or index == 3:
        with open("../../data/domain1_train.json") as f:
            for line in f:
                # read line by line
                data = json.loads(line)

                # add values
                text.append(data["text"])
                label.append(data["label"])

    if index == 2 or index == 3:
        with open("../../data/domain2_train.json") as f:
            for line in f:
                # read line by line
                data = json.loads(line)

                # add values
                text.append(data["text"])
                label.append(data["label"])

    # print(f"Domain{index} length:", len(text))
    
    return text, label


In [58]:
print("Domain1 length:", len(load_data(1)[0]))
print("Domain2 length:", len(load_data(2)[0]))
print("Domain1&2 length:", len(load_data(3)[0]))

Domain1 length: 19500
Domain2 length: 14900
Domain1&2 length: 34400


## 2. Data Preprocessing

#### Create sample data for making vertor space

In [59]:
def vector_sample(text):
    # Number of words: 5000
    sample = [np.arange(5000).tolist()]

    # For n-gram
    sample += text

    return sample

In [60]:
# Data domain
d = 1
print(f"sample size for D{d}:", len(vector_sample(load_data(d)[0])))

sample size for D1: 19501


#### Vectorize the data

In [61]:
from sklearn.feature_extraction.text import CountVectorizer

def toStr(n):
   return str(n)

def count_vec(vector_sample, text, n_features):
   # Create a Vectorizer Object
   vectorizer = CountVectorizer(preprocessor= toStr, analyzer="word", token_pattern=r"(?u)\b\w+\b", ngram_range=(1, 3), max_features=n_features)

   vectorizer.fit(vector_sample)

   # Printing the identified Unique words along with their indices
   # print("Vocabulary: ", vectorizer.vocabulary_)

   # Encode the Document
   vector = vectorizer.transform(text)

   # Summarizing the Encoded Texts
   # print("Encoded Document is:")
   # print(vector.toarray())

   return vector


In [62]:
# Data domain
d = 1
count_vec(vector_sample(load_data(d)[0]), load_data(d)[0], 10000).toarray().shape

(19500, 10000)

## 3. Set Parameters & Train the models

In [63]:
DOMAINS = range(1, 4)
N_FEATURES = range(5000, 20000, 5000)
T_DEPTHS = range(1, 10, 2)


In [64]:
def BoW_training(domain, features, t_depth):
    # BoW
    d = domain
    f = features
    t = t_depth
    text, label = load_data(d)
    sample = vector_sample(text)
    X = count_vec(sample, text, f).toarray()
    y = np.array(label).ravel()

    clf = RandomForestClassifier(max_depth=t, random_state=0)
    clf.fit(X, y)

    return clf, X, y

## 4. Validation

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import pandas as pd

n = 10
cv_scores = []
r2_scores = []
c_matrix = []


for d in (2,3):#DOMAINS:
    for f in N_FEATURES:
        for t in T_DEPTHS:
            # For BoW
            clf, X, y = BoW_training(d, f, t)

            print(f"For Domain{d} & {f} Features & {t} Depth:")
            print("Shape of X:", X.shape)
            print("Shape of y:", y.shape)
    
            # r2 score
            y_pred = clf.predict(X)
            r2_scores.append(r2_score(y, y_pred))
            print("R2 Score\n", r2_scores[-1])

            # Confusion matrix
            c_matrix.append(confusion_matrix(y, y_pred))
            print("Confusion Matirx\n", c_matrix[-1])

            # Cross-validation score
            cv_scores.append(cross_val_score(clf, X, y, cv=n))
            print("CV Score\n", cv_scores[-1], "\n")

            # To csv
            cv_result = pd.DataFrame([r2_scores[-1]]+[sum(cv_scores[-1])/n]+cv_scores[-1].tolist(), index=["r2", "cv_avg"]+[f"{i}" for i in range(1,11)], columns=["scores"])
            cv_result.to_csv(f"../../data/results/RF/RF-CV-D{d}_F{f}T{t}.csv")
            cm_result = pd.DataFrame({"tn": [c_matrix[-1][0][0]], "fn": [c_matrix[-1][1][0]], "tp": [c_matrix[-1][1][1]], "fp": [c_matrix[-1][0][1]]})
            cm_result.to_csv(f"../../data/results/RF/RF-CM-D{d}_F{f}T{t}.csv")


## 5. Visualisation

In [71]:
y_pred
y

array([1, 1, 1, ..., 0, 0, 0])